<p><img src="https://mcd.unison.mx/wp-content/themes/awaken/img/logo_mcd.png" width="100" align="center"></p>

# Clasificación de documentos 

## Ingeniería de Características 

### Maestría en Ciencia de Datos
### Universidad de Sonora

#### Julio Waissman Vilanova (julio.waissman@unison.mx)


En esta libreta vamos a aplicar las herramientas necesarias para el tratamiento de documentos cuando la secuencia de *tokens* no es importante, al menos en el largo plazo. Estas técnicas tienen la ventaja de ser más rápidas de implementar, y para la clasificación de documentos suele dar buenos resultados. Es por esta razón que un método como este *siempre* debe de aplicarse en un principio como un resultado mínimo aceptable. 

Vamos a revisar en primer lugar los métodos de extracción de características. En segundo lugar, aplicaremos os métodos usuales para la clasificación de documentos. Por último, vamos a discutir sobre el uso de representaciones densas, así como el uso de métodos basados en aprendizaje profundo para la clasificación de documentos.

Vamos a procesar la información sobre el problema de análisis de sentimiento de *TASS 2015*

In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import xml.etree.ElementTree as et
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (20, 12)

# Para poder convertir los tópicos en un problema de multiple clasificación
from ast import literal_eval 


def lee_datos_TASS2015(archivo, etiquetado=True):
    arbol = et.parse(archivo)
    raiz = arbol.getroot()
    data_dic = []
    for tweet in raiz.iter('tweet'):
        contenido = tweet.find('content').text
        if contenido is not None:
            data_dic.append({
                'texto': contenido,
                'id': tweet.find('tweetid').text,
                'usuario': '@' + tweet.find('user').text,
                'fecha': tweet.find('date').text,               
            })
            if etiquetado:
                data_dic[-1]['polaridad'] = tweet.find('sentiments')[0].find('value').text
                data_dic[-1]['tópicos'] = '[' + ', '.join(
                    ['"' + t.text + '"' for t in tweet.find('topics')]) + ']'
    return pd.DataFrame.from_dict(data_dic)

df_train = lee_datos_TASS2015("general-tweets-train-tagged.xml") 
df_test = lee_datos_TASS2015("general-tweets-test.xml", False)

y ahora vamos a limpiar los datos usando expresiones regulares y steaming

In [2]:
import re
import nltk

usuarios_re = re.compile(r"@[\w\d]+")
hashtags_re = re.compile(r"#[\w\d]+")
email_re = re.compile(r"\b[\w][\w\.-]*@\w[\w\.-]*\.[a-zA-Z]{2,6}\b")
url_re = re.compile(r"\b(\w+:\/{2})?[\d\w-]+(\.[\d\w-]+)+(/\S+)*\b") 
remplaza_por_espacios_re = re.compile('[\n/(){}\[\]\|@,;\.]')
simbolos_a_eliminar_re = re.compile('[^\d\w #+_]')
palabras_paro = nltk.corpus.stopwords.words('spanish')

def prepara_texto(texto):
    text = texto.lower()
    
    # Codificaciones (problemas con UTF-8, latin1, etc...)
    text = re.sub(r'\\\\', r'\\', text)
    text = re.sub(r'\\\\', r'\\', text)
    text = re.sub(r'\\x\w{2,2}', ' ', text)
    text = re.sub(r'\\u\w{4,4}', ' ', text)
    text = re.sub(r'\\n', ' . ', text)

    # Cambia e_mails, urls y usuarios por palabra clave
    text = re.sub(email_re, '_EMAIL_', text)
    text = re.sub(url_re, '_URL_', text)
    text = re.sub(usuarios_re, '_USR_', text)
    text = re.sub(hashtags_re, '_HASHTAG_', text)
    
    # Las palabras con letras repetidas más de 3 veces 
    # (dos veces por las personas que abusan demasiado)
    text = re.sub(r'([a-zA-Z])\1\1+(\w*)', r'\1\1\2', text)
    text = re.sub(r'([a-zA-Z])\1\1+(\w*)', r'\1\1\2', text)
    
    # Elimina simbolos
    text = re.sub(remplaza_por_espacios_re, ' ', text)
    text = re.sub(simbolos_a_eliminar_re, '', text)
    
    # Palabras de paro
    text = [palabra for palabra in text.split(' ') if palabra not in palabras_paro]
    return ' '.join(text)

tokenizer = nltk.tokenize.WhitespaceTokenizer()
stemmer = nltk.stem.snowball.SpanishStemmer()
def stem_texto(texto):
    return ' '.join(stemmer.stem(token) for token in tokenizer.tokenize(texto))

df_train['texto procesado'] = df_train['texto'].apply(prepara_texto)
df_train['texto stem'] = df_train['texto procesado'].apply(stem_texto)
display(df_train.head())

df_test['texto procesado'] = df_test['texto'].apply(prepara_texto)
df_test['texto stem'] = df_test['texto procesado'].apply(stem_texto)
display(df_test.head())


,texto,id,usuario,fecha,polaridad,tópicos,texto procesado,texto stem
0,"Salgo de #VeoTV , que día más largoooooo...",142389495503925248,@ccifuentes,2011-12-02T00:47:55,NONE,"[""otros""]",salgo _HASHTAG_ día largoo,salg _hashtag_ dia largo
1,@PauladeLasHeras No te libraras de ayudar me/n...,142389933619945473,@CarmendelRiego,2011-12-02T00:49:40,NEU,"[""otros""]",_USR_ libraras ayudar besos gracias,_usr_ libr ayud bes graci
2,@marodriguezb Gracias MAR,142391947707940864,@CarmendelRiego,2011-12-02T00:57:40,P,"[""otros""]",_USR_ gracias mar,_usr_ graci mar
3,"Off pensando en el regalito Sinde, la que se v...",142416095012339712,@mgilguerrero,2011-12-02T02:33:37,N+,"[""política"", ""economía""]",off pensando regalito sinde va sgae van corru...,off pens regalit sind va sga van corrupt inten...
4,Conozco a alguien q es adicto al drama! Ja ja ...,142422495721562112,@paurubio,2011-12-02T02:59:03,P+,"[""otros""]",conozco alguien q adicto drama ja ja ja suena d,conozc algui q adict dram ja ja ja suen d


,texto,id,usuario,fecha,texto procesado,texto stem
0,"Portada 'Público', viernes. Fabra al banquillo...",142378325086715906,@jesusmarana,2011-12-02T00:03:32,portada público viernes fabra banquillo orde...,port public viern fabr banquill orden suprem w...
1,"Grande! RT @veronicacalderon ""El periodista es...",142379080808013825,@EvaORegan,2011-12-02T00:06:32,grande rt _USR_ periodista alguien quiere cont...,grand rt _usr_ period algui quier cont realid ...
2,Gonzalo Altozano tras la presentación de su li...,142379173120442368,@LosadaPescador,2011-12-02T00:06:55,gonzalo altozano tras presentación libro 101 e...,gonzal altozan tras present libr 101 español d...
3,"Mañana en Gaceta: TVE, la que pagamos tú y yo,...",142379815708803072,@mgilguerrero,2011-12-02T00:09:28,mañana gaceta tve pagamos culpa becaria fals...,mañan gacet tve pag culp becari fals inform ci...
4,Qué envidia “@mfcastineiras: Pedro mañana x la...,142381190123499520,@pedroj_ramirez,2011-12-02T00:14:55,envidia _USR_ pedro mañana x mañana voy paris ...,envidi _usr_ pedr mañan x mañan voy paris alme...


In [27]:
# Aqui se puede escoger entre 'texto', 'texto procesado' y 'texto stem'
columna = 'texto stem'
x_train = df_train[columna].values
x_test = df_test[columna].values

y_polaridad = df_train['polaridad'].values
y_topico = df_train['tópicos'].apply(literal_eval).values

df_train.polaridad.unique()

array(['NONE', 'NEU', 'P', 'N+', 'P+', 'N'], dtype=object)

## 3.1. Extracción de características


La extracción de características, sin un método de entrenamiento, solamente puede hacerse con una matriz de características dispersas de dimensión $N \times V$, donde $N$ es el número de documentos y $V$ es el tamaño del vocabulario. Esto es debido al desconocimiento de las relaciones de las palabras entre sí. Esta no solo es una matriz gigantésca, si no que aparte la gran mayoría de las entradas de la matriz van a ser cero.

El método de base es el conocido como *bolsa de palabras (BOW por Bag of words)*, en el cual se asigna un valor de 1 a cada palabra que se encuentre *al menos* una vez en un documento. Como el uso de palabras solas implica que no existe ninguna relación entre palabras, la ampliación del vocabulario a bigramas, trigramas, o $n$-gramas de cualquier orden, permite conservar en cierta forma las relaciones secuenciales más simples, pero a costa de un incremento en forma exponencial de nuestro vocabulario. Vamos a extraer características tanto con unigramas únicamente como con bigramas y unigramas. ¿Qué pasa si se mueven algunos parámetros como `min_df` o `token_pattern`?


In [28]:
from sklearn.feature_extraction.text import CountVectorizer

bog_unigramas_vectorizer = CountVectorizer(
    analyzer='word',        #  Separa por palabras (puede ser por caracteres)
    binary=True,            #  BOG (si False, entonces cuenta numero de ocurrencias por documento)
    ngram_range=(1,1),      #  n-gramas (min, max) a utilizarse, en este caso solo unigramas
    token_pattern=r'(\S+)', #  Patron de tokenización. Por default solo letras y al menos dos letras
    min_df=5,               #  Número de documentos (o proporción) mínimo donde un termino valido aparece
    max_df=0.9,             #  Número de documentos (o proporción) máximo donde un termino válido aparece
)

bog_bigramas_vectorizer = CountVectorizer(
    binary=True, 
    ngram_range=(1,2), 
    token_pattern=r'(\S+)', 
    min_df=5, 
    max_df=0.9
)

x_train_bow1 = bog_unigramas_vectorizer.fit_transform(x_train)
x_text_bow1 = bog_unigramas_vectorizer.transform(x_test)

x_train_bow2 = bog_bigramas_vectorizer.fit_transform(x_train)
x_test_bow2 = bog_bigramas_vectorizer.transform(x_test)

print("El objeto x_train_bow1 es del tipo{}".format(type(x_train_bow1)))
print("La dimensión de x_train_bow1 es {}".format(x_train_bow1.shape))
print("La dimensión de x_train_bow2 es {}".format(x_train_bow2.shape))

El objeto x_train_bow1 es del tipo<class 'scipy.sparse.csr.csr_matrix'>
La dimensión de x_train_bow1 es (7218, 2133)
La dimensión de x_train_bow2 es (7218, 2672)


[TF-IDF](https://en.wikipedia.org/wiki/Tf–idf) (*term frequency–inverse document frequency*) es una medida de la importancia discriminante de una palabra en un documento perteneciente a un corpus. Esta cantidad se forma por dos componentes:

* *TF* (frecuencia del término), describe qué tanto se emplea un término en un documento. La lógica de este componente es que mientras más veces aparece un término en un documento, más representativo es del mismo. Existen diversa maneras de calcular este valor, siendo la más simple el conteo directo del término en el documento. La mayoría de las opciones, sin embargo, utilizan alguna variante basada en la frecuencia relativa del término en el documento. 

$$
tf(t, d) = \frac{\textrm{Número de veces que aparece el término }t\textrm{ en el documento }d}
{\textrm{Número de términos en el documento }d} 
$$

* *IDF* (frecuencia inversa del documento), representa la frecuencia con que es empleado el término en el corpus. Un término que es utilizado con mucha frecuencia en el contexto, es poco  discriminante. La forma básica de calcular esta cantidad es como el inverso de la frecuencia de documentos en que aparece el término (escalado logarítmicamente).

$$
idf(t, D) = \log\frac{ 1 + \textrm{Número de documentos en el corpus }D}
{1 + \textrm{Número de documentos en el corpus }D\textrm{ donde aparece el término t}} + 1
$$

El valor de TF para un término/palabra es específico para cada documento, mientras que IDF es un valor global del término en el corpus. El valor de TF-IDF se obtiene multiplicando los valores de TF e IDF:

$$
tf\_idf(t, d, D) = tf(t, d)\cdot idf(t, D)
$$
y al final cada renglón es normalizado respecto a todos los términos, de forma que cada documento sea representado por un vector de características con norma unitaria (lo que es particularmente interesante para el entrenamiento de muchos modelos de aprendizaje automático).

Este método de extracción de características es una modificación al BOG y por lo tanto, la clase para el vectorizador TF-IDF es básicamente la misma. 

In [29]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_unigramas_vectorizer = TfidfVectorizer(
    ngram_range=(1,1),      #  n-gramas (min, max) a utilizarse, en este caso solo unigramas
    token_pattern=r'[a-zñáéíóúü]+', #  Patron de tokenización. Por default solo letras y al menos dos letras
    min_df=5,               #  Número de documentos (o proporción) mínimo donde un termino valido aparece
    max_df=0.9,             #  Número de documentos (o proporción) máximo donde un termino válido aparece
    norm='l2'               #  Norma del vector de cada palabra, lo mñas típico es la euclidiana ('l2')
)

tfidf_bigramas_vectorizer = TfidfVectorizer(
    ngram_range=(1,2), 
    token_pattern=r'[a-zñáéíóúü]+',
    min_df=5, 
    max_df=0.9
)

x_train_tfidf1 = tfidf_unigramas_vectorizer.fit_transform(x_train)
x_text_tfidf1 = tfidf_unigramas_vectorizer.transform(x_test)

x_train_tfidf2 = tfidf_bigramas_vectorizer.fit_transform(x_train)
x_test_tfidf2 = tfidf_bigramas_vectorizer.transform(x_test)

print("El objeto x_train_tfidf1 es del tipo{}".format(type(x_train_tfidf1)))
print("La dimensión de x_train_tfidf1 es {}".format(x_train_tfidf1.shape))
print("La dimensión de x_train_tfidf2 es {}".format(x_train_tfidf2.shape))

El objeto x_train_tfidf1 es del tipo<class 'scipy.sparse.csr.csr_matrix'>
La dimensión de x_train_tfidf1 es (7218, 2071)
La dimensión de x_train_tfidf2 es (7218, 2626)


A partir del objeto vectorizador generado (en esta caso `tfidf_bigramas_vectorizer`) se puede obtener información adicional. En particular es importante conocer a que token corresponde cada una de las columnas y viceversa.

In [30]:
vocabulario = tfidf_bigramas_vectorizer.get_feature_names()

for i in [1, 3, 100, 1000, 2000]:
    print('La palabra en la posición {} es "{}"'.format(i, vocabulario[i]))
# display(vocabulario)

print('\nLa palabra "amor" se encuentra en la posición {}'.format(tfidf_bigramas_vectorizer.vocabulary_.get('amor')))

numdoc = np.random.randint(0, 1500)
a = x_train_tfidf2[numdoc, :].toarray().ravel()
mejores = [(vocabulario[i], a[i]) for i in a.argsort()[-1:-4:-1]]

print('\nEl documento {} es:\n\t"{}"'.format(numdoc, x_train[numdoc]))
print('y los 3 tokens con mayor valor de tf-idf son:')
for (token, tfidf) in mejores:
    print("\t{}\t\tvalor de ttf-idf: {:06.4f}".format(token, tfidf))

La palabra en la posición 1 es "aa"
La palabra en la posición 3 es "abc"
La palabra en la posición 100 es "alumn"
La palabra en la posición 1000 es "for"
La palabra en la posición 2000 es "raul"

La palabra "amor" se encuentra en la posición 114

El documento 442 es:
	"capaz mejor cancion tant vec oi cant mejor maner record quier _url_"
y los 3 tokens con mayor valor de tf-idf son:
	mejor		valor de ttf-idf: 0.4522
	capaz		valor de ttf-idf: 0.3744
	cancion		valor de ttf-idf: 0.3290


En general, para la clasificación de textos el método de TF-IDF da mejores resultados que BOW, y su complejidad computacional es muy similar. 

## 3.2. Clasificación de documentos

En esta sección vamos a mostrar dos formas diferentes de clasificar documentos, utilizando los métodos disponibles en *sklearn*: Una para una clasificación en la que cada documento del conjunto de aprendizaje se encuentra etiquetado en una sola clase (como es el caso de la polaridad, para el análisis de sentimientos), y el caso en el cual cada documento puede estar asignado a una o más etiquetas (como es el caso de los tópicos).

Al estar fuera del alcance de este curso la discusión sobre los métodos de aprendizaje supervisado, vamos aretener solamente uno de los métodos más clásicos (regresión logística). Les recomiendo explorar otros métodos, en particular las máquinas de vectores de soporte para clasificación y los bosques aleatorios. Estos dos métodos suelen obtener muy buenos resultados en los diferentes *benchmarks* y competencias.

### 3.2.1 Análisis de sentimientos (polaridad)

Comencemos por el principio, que es separar los datos de entrenamiento de los de validación 

In [31]:
from sklearn.model_selection import train_test_split

x_entrena, x_valida, y_entrena, y_valida = train_test_split(
    x_train_tfidf2, 
    y_polaridad, 
    test_size=0.1, 
    random_state=10
)
print('Tamaño del conjunto de entrenamiento = {}. \
       Tamaño del conjunto de validación = {}.'.format(x_entrena.shape[0], x_valida.shape[0]))

Tamaño del conjunto de entrenamiento = 6496.        Tamaño del conjunto de validación = 722.


Ahora vamos a generar un objeto clasificador, con un [clasificador logístico](http://scikit-learn.org/stable/modules/linear_model.html#logistic-regression), y realizamos el entrenamiento. Para más información sobre la regresión logística un buen tutorial se puede encontrar [aquí (capítulo 2, pero el capítulo 1 tambien está muy bueno](http://cs229.stanford.edu/notes/cs229-notes1.pdf)).

In [32]:
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression(
    penalty='l2',          # Norma a seguir en la optimización ('l2' o 'l1')
    C=1.0,                 # Inverso a la constante de regularización, para mejorar la generalización
    random_state=1,        # Solo para poder tener repetibilidad (muy útil para debuguear)
    multi_class='ovr'      # Para más de dos clases ('ovr' para one-vs-all, 'multinomial' para softmax)
)

clf.fit(x_entrena, y_entrena)

/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='ovr', n_jobs=None, penalty='l2', random_state=1,
                   solver='warn', tol=0.0001, verbose=0, warm_start=False)

Y ahora vamos a ver los resultados, de acuerdo a los [indicadores clásicos](https://en.wikipedia.org/wiki/Precision_and_recall).

In [33]:
from sklearn.metrics import classification_report, accuracy_score

def reporte(clasificador, x, t, labels=None):
    y = clasificador.predict(x)
    print("\nPorcentaje de acierto: {}".format(accuracy_score(t, y)))
    print("\nPrecisión, recall y f1-score")
    print(classification_report(t, y, target_names=labels))    

print("\n\nPara los datos de entrenamiento\n" + 40*"=")
reporte(clf, x_entrena, y_entrena)
print("\n\nPara los datos de validación\n" + 40*"=")
reporte(clf, x_valida, y_valida)



Para los datos de entrenamiento

Porcentaje de acierto: 0.6617918719211823

Precisión, recall y f1-score
              precision    recall  f1-score   support

           N       0.59      0.75      0.66      1201
          N+       0.78      0.55      0.64       764
         NEU       0.84      0.20      0.33       597
        NONE       0.65      0.83      0.73      1328
           P       0.70      0.47      0.57      1109
          P+       0.67      0.83      0.74      1497

    accuracy                           0.66      6496
   macro avg       0.70      0.60      0.61      6496
weighted avg       0.68      0.66      0.64      6496



Para los datos de validación

Porcentaje de acierto: 0.4196675900277008

Precisión, recall y f1-score
              precision    recall  f1-score   support

           N       0.33      0.50      0.40       134
          N+       0.35      0.20      0.26        83
         NEU       0.38      0.07      0.12        73
        NONE       0.50      

Los [resultados reportados](http://ceur-ws.org/Vol-1397/overview.pdf) con el conjunto de prueba (del que no disponemos los datos) se encuentran distribuidos alrededor del 0.5 de porcentaje de acierto. Es interesante notar como nuestros resultados no son tan alejados (en un primer intento) de los presentados en trabajos que hacen uso de tecnicas extremadamente sofisticadas, como son las [redes neuronales profundas recurrentes](http://ceur-ws.org/Vol-1397/lys.pdf). Sin embargo, no estamos conformes con los resultados y debemos de hacer una mejor clasificación, lo que nos lleva a la *tarea 1*.



Por otro lado, es importante la interpretabilidad del modelo. Dado que en la regresión logística se agrega la información de cada una de las características de forma lineal, es posible ver a los coeficientes como una medida de peso de la importancia (positiva y negativa) que tienen cada uno de los *tokens* para cada una de las etiquetas. Veamos cuales son las palabras con más peso positivo y con más peso negativo para una etiqueta.

In [39]:
def tokens_principales(clasificador, clase, vocabulario):
    clase_ind = clasificador.classes_.tolist().index(clase)
    pesos = clasificador.coef_[clase_ind,:]
    indices = pesos.argsort()
    mejores = [(vocabulario[i], pesos[i]) for i in indices[-1:-11:-1]]
    peores = [(vocabulario[i], pesos[i]) for i in indices[:10]]
    
    return mejores, peores

clase_sel = 'N+'
mejores, peores = tokens_principales(clf, clase_sel, vocabulario)

print("Las clases disponibles son: {}".format(clf.classes_.tolist()))
print("Los tokens más positivos para la clase {} son:".format(clase_sel))
for (t, w) in mejores:
    print("\t- {:25}(peso:{:6.4})".format(t,w))
print("Los tokens más negativos para la clase son:")
for (t, w) in peores:
    print("\t- {:25}(peso:{:6.4})".format(t,w))

Las clases disponibles son: ['N', 'N+', 'NEU', 'NONE', 'P', 'P+']
Los tokens más positivos para la clase N+ son:
	- deficit                  (peso: 3.423)
	- recort                   (peso: 3.394)
	- corrupcion               (peso: 3.275)
	- par                      (peso: 3.088)
	- denunci                  (peso: 2.115)
	- vergüenz                 (peso: 1.986)
	- crisis                   (peso: 1.938)
	- desp                     (peso: 1.866)
	- muert                    (peso: 1.853)
	- tram                     (peso: 1.837)
Los tokens más negativos para la clase son:
	- buen                     (peso:-2.874)
	- graci                    (peso:-2.001)
	- usr                      (peso:-1.716)
	- port                     (peso:-1.608)
	- hoy                      (peso:-1.573)
	- noch                     (peso:-1.362)
	- hashtag                  (peso:-1.338)
	- quier                    (peso:-1.336)
	- url                      (peso:-1.276)
	- feliz                    (peso:-1.252)


Por último, veamos que encuentra con el conjunto de prueba, revisando algunos ejemplos

In [ ]:
y_test = clf.predict(x_test_tfidf2)

for i in [1, 10, 100, 500]:
    print("Twwet:\t {}".format(df_test.loc[i,'texto']))
    print('Polaridad estimada: {}'.format(y_test[i]))

### 3.2.2. Clasificación de tópicos

Ahora vamos a realizar la clasificación de tópicos, la cual es una tarea más dificil que el análisis de sentimientos, al estár más clases involucradas, y al tener varias etquetas por cada documento. Para esto vamos a utilizar los métodos del módulo *sklearn*. Vamos a preparar los datos para realizar la tarea:

In [36]:
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.multiclass import OneVsRestClassifier

# Se procesa las etiquetas en forma de multietiquetas
topico_count = {}
for y in y_topico:
    for topico in y:
        topico_count[str(topico)] = topico_count.setdefault(topico, 0) + 1    
mlb = MultiLabelBinarizer(classes=sorted(topico_count.keys()))
y_topico_mlb = mlb.fit_transform(y_topico)

# Se separan los datos en conjunto de entrenamiento y validación
x_ent_mlb, x_val_mlb, y_ent_mlb, y_val_mlb = train_test_split(
    x_train_tfidf2, 
    y_topico_mlb, 
    test_size=0.1, 
    random_state=10
)

# Se crea un clasificador para multiples etiquetas
clf_mlb = OneVsRestClassifier(
    LogisticRegression(
        C=5.0,
        penalty='l2'
    )
)

# Se entrena el clasificador
clf_mlb.fit(x_ent_mlb, y_ent_mlb)

# Se revisa los indices de desempeño
print("\n\nPara los datos de entrenamiento\n" + 40*"=")
reporte(clf_mlb, x_ent_mlb, y_ent_mlb, mlb.classes)
print("\n\nPara los datos de validación\n" + 40*"=")
reporte(clf_mlb, x_val_mlb, y_val_mlb, mlb.classes)

/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)




Para los datos de entrenamiento

Porcentaje de acierto: 0.6134544334975369

Precisión, recall y f1-score
                 precision    recall  f1-score   support

           cine       0.92      0.30      0.46       220
       deportes       1.00      0.25      0.41       102
       economía       0.94      0.70      0.81       855
entretenimiento       0.91      0.56      0.69      1531
         fútbol       0.98      0.43      0.59       225
     literatura       1.00      0.35      0.52        89
         música       0.97      0.51      0.67       505
          otros       0.87      0.76      0.81      2086
       política       0.94      0.88      0.91      2802
     tecnología       0.97      0.30      0.46       191

      micro avg       0.92      0.70      0.80      8606
      macro avg       0.95      0.50      0.63      8606
   weighted avg       0.92      0.70      0.78      8606
    samples avg       0.79      0.72      0.74      8606



Para los datos de validación

Por

/opt/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)


Y para terminar la sección, revisemos algunos tweets del conjunto de prueba

In [37]:
y_test_mlb = clf_mlb.predict(x_test_tfidf2)
y_test_topico = mlb.inverse_transform(y_test_mlb)

for i in [1, 10, 100, 500, 1000, 2000]:
    print("Twwet:\t {}".format(df_test.loc[i,'texto']))
    print('Tópicos estimados: {}'.format(y_test_topico[i]))



Twwet:	 Grande! RT @veronicacalderon "El periodista es alguien que quiere contar la realidad, pero no vive en ella" via @galtares
Tópicos estimados: ('entretenimiento',)
Twwet:	 Definitivamente, creo que me he resfriado. Con este tiempo de locos que ha estado haciendo estos meses, ahora toca las consecuencias.
Tópicos estimados: ('política',)
Twwet:	 No hubo tiempo para sacar a Franco del Valle de los Caídos pero sí lo ha habido para ampliar la base militar de Rota.
Tópicos estimados: ('política',)
Twwet:	 Curiosa la evolución del twitter de @_rubalcaba_ ¿Seguimos en jornada de reflexión? http://t.co/i1btoFHo
Tópicos estimados: ('entretenimiento',)
Twwet:	 Venga y vamos a apuntarnos todos RT @FotosYTuits: Ya hay fecha!!!! Marcar el jueves 15 de diciembre #fyt06 http://t.co/KgXvI7fI
Tópicos estimados: ()
Twwet:	 Vuelvo a tweeter,he estado ausente una semana,me gusta más el mundo de la presencia física,el apretón de manos del amigo,la palabra cálida..
Tópicos estimados: ('otros',)
